# Transform

## Import

In [149]:
import pandas as pd
import numpy as np
import os
import json

In [150]:
# Pfad
TEMPLATE_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "_templates"))

In [151]:
# Lade die Pickle-Dateien
partei_df = pd.read_pickle(os.path.join(TEMPLATE_PATH, "parteien.pickle"))
wahlen = pd.read_pickle(os.path.join(TEMPLATE_PATH, "politbarometer.pickle"))

## Analyse

In [152]:
print(partei_df.shape, wahlen.shape)

(120558, 4) (568, 8)


In [153]:
partei_df.head()

,count,word,date,paper
0,1,cdu,2021-04-01,sz
1,1,cdu,2021-04-01,sz
2,2,cdu,2021-04-01,zeit
3,1,cdu,2021-04-01,zeit
4,1,grünen,2021-04-01,faz


In [154]:
wahlen.head()

,date,CDU,SPD,Grüne,FDP,Linke,AfD,BSW
0,1991-02-15,43,35,7,10.0,2.0,NaN,NaN
1,1991-03-15,42,37,7,10.0,2.0,NaN,NaN
2,1991-04-26,41,38,7,10.0,2.0,NaN,NaN
3,1991-05-24,41,39,6,10.0,2.0,NaN,NaN
4,1991-06-14,40,40,6,9.0,2.0,NaN,NaN


In [155]:
# Eine Kopie der Daten erstellen
wahlen = wahlen.copy()
wordcount = partei_df.copy()

## Wordcount df's erstellen

In diesem Code werden die gesuchten Parteinamen ab einem bestimmten Datum gefiltert, aggregiert und in verschiedene Zeitformate umgewandelt, um eine übersichtliche Auswertung zu ermöglichen.

In [156]:
# Einschränken auf Daten ab 2021-04-01 und Index zurücksetzen
wordcount = wordcount[wordcount['date'] >= '2021-04-01'].reset_index(drop=True)
wordcount = wordcount[wordcount["date"] <= "2024-12-01"].reset_index(drop=True)

# Gruppieren nach Datum und Wort sowie Aufsummieren der 'count'-Werte
wordcount = wordcount.groupby(['date', 'word']).agg({'count': 'sum'}).reset_index()

# Konvertieren der 'date'-Spalte in ein Datumsformat
wordcount["date"] = pd.to_datetime(wordcount["date"])

# Hinzufügen einer Spalte für das Jahr und den Monat (Format 'YYYY-MM')
wordcount["month"] = wordcount["date"].dt.strftime('%Y-%m')

# Hinzufügen einer Spalte für das Quartal
wordcount["quarter"] = wordcount["date"].dt.to_period('Q')

# Ermitteln der Gesamtanzahl pro Wort
word_counts = wordcount.groupby("word")["count"].sum()

In [157]:
wordcount.head()

,date,word,count,month,quarter
0,2021-04-01,afd,8,2021-04,2021Q2
1,2021-04-01,cdu,43,2021-04,2021Q2
2,2021-04-01,fdp,2,2021-04,2021Q2
3,2021-04-01,grünen,20,2021-04,2021Q2
4,2021-04-01,linke,9,2021-04,2021Q2


Hier werden die Parteien definiert, die in der Visualisierung zusammen dargestellt werden sollen

In [158]:
# Liste der Wörter, die behalten werden sollen
words_to_keep = ["grünen", "afd"]
wordkeep = ["spd", "cdu"]
# Filtern des DataFrames, um nur die gewünschten Wörter zu behalten
kleine_parties = wordcount[wordcount["word"].isin(words_to_keep)]
große_parties = wordcount[wordcount["word"].isin(wordkeep)]

## Wahlen (Politbarometer) df erstellen

In diesem Code werden die Barometerdaten auf einen bestimmten Zeitraum eingeschränkt, ihre Spalten vereinheitlicht und zusätzliche Zeitintervalle für die Analyse hinzugefügt.

In [159]:
# Konvertieren der 'date'-Spalte in ein Datumsformat
wahlen["date"] = pd.to_datetime(wahlen["date"])

# Daten auf Zeitraum vom 01.04.2021 bis 01.12.2024 beschränken
wahlen = wahlen[wahlen["date"] >= "2021-04-01"].reset_index(drop=True)
wahlen = wahlen[wahlen["date"] <= "2024-12-01"].reset_index(drop=True)

# Spaltennamen vereinheitlichen und neue Zeitspalten hinzufügen
wahlen.columns = wahlen.columns.str.lower()
wahlen["quarter"] = wahlen["date"].dt.to_period("Q").astype(str)
wahlen["month"] = wahlen["date"].dt.to_period("M").astype(str)

# Konvertieren der CDU-Spalte in Integer
wahlen["cdu"] = wahlen["cdu"].astype(int)

# Wandle alle spezifischen Spalten von float zu integer um
wahlen[['cdu', 'spd', 'grüne', 'fdp', 'linke', 'afd', 'bsw']] = wahlen[['cdu', 'spd', 'grüne', 'fdp', 'linke', 'afd', 'bsw']].fillna(0).astype(int)

# Länge jeder Spalte ausgeben
for key, value in wahlen.items():
    print(f'Länge von {key}:', len(value))


Länge von date: 72
Länge von cdu: 72
Länge von spd: 72
Länge von grüne: 72
Länge von fdp: 72
Länge von linke: 72
Länge von afd: 72
Länge von bsw: 72
Länge von quarter: 72
Länge von month: 72


In [160]:
wahlen.head()

,date,cdu,spd,grüne,fdp,linke,afd,bsw,quarter,month
0,2021-04-16,31,14,21,9,7,11,0,2021Q2,2021-04
1,2021-05-07,25,14,26,10,7,11,0,2021Q2,2021-05
2,2021-05-21,24,14,25,11,7,11,0,2021Q2,2021-05
3,2021-06-10,28,15,22,10,7,11,0,2021Q2,2021-06
4,2021-06-25,29,14,22,10,7,10,0,2021Q2,2021-06


## json-Datei erstellen fuer d3 visualisierung


In [161]:
df = partei_df.copy()

# Datum umwandeln und sortieren
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(by=["word", "date"])

# Kumulative Summe berechnen
df["cumulative_count"] = df.groupby("word")["count"].cumsum()

# Datum in JSON-kompatibles Format umwandeln
df["date"] = df["date"].dt.strftime("%Y-%m-%d")

# Gruppieren und JSON-strukturierte Daten erstellen
grouped_data = df.groupby("word").apply(
    lambda group: group[["date", "cumulative_count"]].to_dict(orient="records")
).to_dict()

## Export

In [162]:
# Df's als pickle speichern
kleine_parties.to_pickle(os.path.join(TEMPLATE_PATH, "kleine.pickle"))
große_parties.to_pickle(os.path.join(TEMPLATE_PATH, "große.pickle"))
wahlen.to_pickle(os.path.join(TEMPLATE_PATH, "wahlen.pickle"))
partei_df.to_pickle(os.path.join(TEMPLATE_PATH, "alle_parteien.pickle"))

In [163]:
# JSON-Pfad fuer d3 Plot 
json_path = os.path.join(TEMPLATE_PATH, "cumulative_parteien.json")

# JSON-Datei speichern
with open(json_path, "w") as f:
    json.dump(grouped_data, f)

print(f"Kumulierte JSON-Daten gespeichert unter: {json_path}")

Kumulierte JSON-Daten gespeichert unter: c:\Users\hp\OneDrive\Desktop\DBU\ads_05\ads_05_studienarbeit\_templates\cumulative_parteien.json
